# Full Analysis of Videos from Experiment on December 2, 2021 with 72 bar Saturation Pressure

Having explored the bubble growth data in `20211202_72bar_eda.ipynb` and improved the method for fitting the bubble growth model to data in `20211202_72bar_compare_fit_methods.ipynb`, I will now analyze the full dataset here.

In [ ]:
%load_ext autoreload
%autoreload 2

# sets plotting font to Arial
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

In [ ]:
# GROWTH MODEL PARAMETERS
growth_fn = bubble.grow
dt = 1E-10 # [s]
R_nuc = 3E-9 # higher value bypasses numerical challenges at nm scale [m]
# make this on Wang group server with `bi_surf`--read README.txt and copy 1k3f_22c.csv
polyol_data_file = '../input/3k2f_31c.csv'
# equation of state data for CO2 at desired temperature 
eos_co2_file = '../input/eos_co2_25-0C.csv'

# FITTING PARAMETERS
frac_lo = 0.95
frac_hi = 1.05
# guess for effective diffusivity constant [m^2/s]
D_lo = 1E-9
D_hi = 15E-9
fit_fn_params = {'err_tol' : 0.003, 'err_fn' : an.calc_abs_sgn_mse}
exp_ratio_tol = 0.01

# PLOTTING PARAMETERS
x_lim = [1E-2, 10]
y_lim = [1, 30]

# USER PARAMETERS
n_fit = 5

# globs list of data directories
data_dir_list = glob.glob(data_dir_tmp)

growth_data_dict = {}

# error tolerances
rms_err_tol = 0.03
sgn_mse_err_tol = 0.003

options = {'single pt' : {'fit_fn' : an.fit_growth_to_pt, 'fit_fn_params': {}},
          'rms' : {'fit_fn' : an.fit_growth_to_pts, 'fit_fn_params' : 
                   {'err_fn' : an.calc_rms_err, 'err_tol' : rms_err_tol}},
          'sgn mse' : {'fit_fn' : an.fit_growth_to_pts, 'fit_fn_params' :
                      {'err_fn' : an.calc_abs_sgn_mse, 'err_tol' : sgn_mse_err_tol}}
          }

for tag in options.keys():
    print('\n\nBeginning analysis for {0:s}.\n\n'.format(tag))
    params = options[tag]
    # fits effective diffusivity D and nucleation time t_nuc
    growth_data = an.fit_D_t_nuc(data_filename, data_dir_list, polyol_data_file, eos_co2_file,
                                 frac_lo, frac_hi, D_lo, D_hi, growth_fn, dt, R_nuc, 
                                 params['fit_fn_params'], exp_ratio_tol, fit_fn=params['fit_fn'],
                                 n_fit=n_fit, x_lim=x_lim, y_lim=y_lim)
    
    # stores results under tag
    growth_data_dict[tag] = growth_data